In [29]:
import dspy
import os
from sql_agent import GenerateSQL, execute_sql, RetrieveSchema

In [3]:
lm = dspy.LM('groq/llama-3.3-70b-versatile', api_key=os.getenv('GROQ_API_KEY'))
dspy.configure(lm=lm)

In [30]:
sql_query_generator = dspy.ReAct(GenerateSQL, tools=[execute_sql])

In [31]:
class GetterSignature(dspy.Signature):
    """Try to get product name and quantity from the user's message
        If Product name or quantity is not provided, ask user to provide it"""
    user_input: str = dspy.InputField(desc="user's message")
    product_name: str = dspy.OutputField(desc="Get Product Name from the user's message")
    quantity: int = dspy.OutputField(desc="Get Quantity from the user's message")
    details: bool = dspy.OutputField(desc="Say whether the product name and quantity is provided or not")

class SomeModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.ChainOfThought(GetterSignature)

    def forward(self, user_input: str):
        extracted = self.predict(user_input=user_input)
        return f"The quantity of {extracted.product_name} is {extracted.quantity}, {extracted.details}"


predictor = SomeModule()


In [32]:
predictor("I want to buy apples")

"The quantity of apples is 0, The product name is provided as 'apples', but the quantity is not provided. Please provide the quantity to complete the order."

In [ ]:
class BuySentiment(dspy.Signature):
    """Tell if user intents to buy anything or not
        If user asks about price, quantity, etc, return False
        Only return True if user asks about buying something"""
    user_input: str = dspy.InputField(desc="user's message")
    buy_intent: bool = dspy.OutputField(desc="True if user intents to buy anything, False otherwise")

class BuyModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.ChainOfThought(BuySentiment)

    def forward(self, user_input: str):
        extracted = self.predict(user_input=user_input)
        return f"User intents to buy: {extracted.buy_intent}"

BUY_INTENT_PREDICTOR = BuyModule()

'User intents to buy: False'

In [ ]:
## CHATBOT CLASS
class SQLChatbot(dspy.Module):
    def __init__(self):
        super().__init__()
        self.history = []

    def forward(self, user_input: str):
        """Handles user queries and maintains conversation history for better results
           We will first check if user wants to buy something or just querying about the database"""

        retrieve_schema = RetrieveSchema()
        query_context = retrieve_schema(user_input)

        response = sql_query_generator(question=user_input, context=query_context, history=self.history)

        # Store conversation history
        self.history.append(f"User: {user_input}")
        self.history.append(f"Bot: {response.answer}")

        return response.answer